In [1]:
import netCDF4 as nc
import numpy as np
import numpy.ma as ma
import sys
import os
import argparse
import xarray as xr 
import matplotlib
%matplotlib inline

# parser = argparse.ArgumentParser(
#         description='generate daily surface C grid velocites files from a years worth of B grid daily velocities ')

# parser.add_argument('--C_grid_spec', metavar='C_grid_spec', default='/archive/net/CM2.6/regrid/CM2p6_half_len.nc',
#                     help='the netcdf file containing the C grid half-face lengths')

# parser.add_argument('--B_vel', type=str, metavar='B_vel', default='/archive/net/02020101_surf_vel.nc',
#                     help='netcdf paths to u and v containing a years worth of B grid daily velocities')

# parser.add_argument('--out', type=str, metavar='out', default='/archive/net/CM2p6_regrid/',
#                     help='the output folder for C grid daily velocity fields')

# parser.add_argument('--year', type=int, metavar='year', default=02020101,
#                     help='the year to be regridded')

# # parser.add_argument('--depth', type=int, metavar='depth', default=0,
# #                     help='the depth at which to regrid')

# args = parser.parse_args()


def roll_values(ds, roll_dict):
    true_coords = ds.coords 
    ds_rolled = ds.roll(**roll_dict)
    for coord in true_coords:
        ds_rolled.coords[coord] = ds.coords[coord]
    return ds_rolled


B_vel_u_path  = '/work/net/02020101.ocean_minibling_field_u.nc'  #args.B_vel
B_vel_v_path  = '/work/net/02020101.ocean_minibling_field_v.nc'  #args.B_vel

C_grid_spec = '/archive/net/CM2.6/regrid/CM2p6_half_len.nc' #args.C_grid_spec
out = '/nbhome/net/CM2p6_regrid/' #args.out
year = 02020101 #args.year
# depth = 0

# if not os.path.exists(out):
#     os.makedirs(out)
#     print "Output Directory:", out


half_len_data = xr.open_dataset(C_grid_spec)
lat_T = half_len_data.grid_y_T[0:2107]
lon_T = half_len_data.grid_x_T[...]
du_Eface_N = half_len_data.ds_21_22_T[0:2107, :].astype(np.float32)
du_Eface_S = half_len_data.ds_20_21_T[0:2107, :].astype(np.float32)

u_B = xr.open_dataset(B_vel_u_path, decode_times=False)
lat_U = u_B.yu_ocean[0:2107]
lon_U = u_B.xu_ocean[...]

#construct the C grid U grid  
xu_c = xr.zeros_like(lon_U)
xu_c.values = np.roll(lon_U.values, 1) #xu_c[i] = lon_U[i-1] 
xu_c[0] = -280.0

xu_c.coords['xu_ocean'].values = xu_c.values
xu_c.attrs['long_name'] = 'ucell longitude on C grid'
xu_c = xu_c.rename({'xu_ocean': 'xu_c'})

yu_c = lat_T
yu_c.coords['grid_y_T'].values = yu_c.values
yu_c = yu_c.rename({'grid_y_T': 'yu_c'})
yu_c.attrs['long_name'] = 'ucell latitude on C grid'

#construct the C grid V grid  
xv_c = lon_T
xv_c.coords['grid_x_T'].values = xv_c.values
xv_c = xv_c.rename({'grid_x_T': 'xv_c'})
xv_c.attrs['long_name'] = 'vcell longitude on C grid'


yv_c = xr.zeros_like(lat_U)
yv_c.values = np.roll(lat_U.values, 1) #xu_c[i] = lon_U[i-1] 

yv_c.coords['yu_ocean'].values = yv_c.values
yv_c = yv_c.rename({'yu_ocean': 'yv_c'})
yv_c.attrs['long_name'] = 'vcell latitude on C grid'

#adds in a row of C grid V points beneath the existing Tcell row 
yv_c[0] = lat_U[0] + lat_U[0] - lat_U[1] 

In [ ]:
u_B_slab.st_ocean[[0, 5, 10, 15, 20, 22, 25]]

In [ ]:
slab_indices = {'yu_ocean': slice(0, 2107), 'st_ocean': [0, 5, 10, 15, 20, 22, 25]}
u_B_slab = u_B.u.isel(**slab_indices).fillna(0.0).astype(np.float32)

u_C_slab = xr.zeros_like(u_B_slab)
u_C_slab = u_C_slab.rename({'xu_ocean': 'xu_c'})
u_C_slab.coords['xu_c'] = xu_c
u_C_slab = u_C_slab.rename({'yu_ocean': 'yu_c'})
u_C_slab.coords['yu_c'] = yu_c
u_C_slab.attrs['long_name'] = 'i-surface current on SW convention C grid'

#set to type to np.float32 
# u_C_slab.values = u_C_slab.values.astype(np.float32)
# u_B_slab.values = u_B_slab.values.astype(np.float32)
# 	for i in range(LON_POINTS):
# 	    for j in range(1,LAT_POINTS):
# 	        u_C[i,j] = ( (du_Eface_N[i-1, j]*u_B[i-1, j] + du_Eface_S[i-1, j]*u_B[i-1, j-1]) \
# 	                    /(du_Eface_N[i, j-1] + du_Eface_S[i, j-1] ) )  

In [ ]:
roll_du_Eface_N = {'grid_x_T': 1, 'grid_y_T': 0}
roll_du_Eface_S = {'grid_x_T': 1, 'grid_y_T': 0}
roll_u_B_N = {'xu_ocean': 1, 'yu_ocean': 0}
roll_u_B_S = {'xu_ocean': 1, 'yu_ocean': 1}

du_Eface_N_roll = roll_values(du_Eface_N, roll_du_Eface_N).values
du_Eface_S_roll = roll_values(du_Eface_S, roll_du_Eface_S).values 
numerat_1 = np.multiply(du_Eface_N_roll, roll_values(u_B_slab, roll_u_B_N).values) 
numerat_2 = np.multiply(du_Eface_S_roll, roll_values(u_B_slab, roll_u_B_S).values) 
denom = du_Eface_N_roll + du_Eface_S_roll

u_C_slab.values = np.divide(numerat_1 + numerat_2, denom)

In [ ]:
v_B = xr.open_dataset(B_vel_v_path, decode_times=False)
slab_indices = {'yu_ocean': slice(0, 2107)}
v_B_slab = v_B.v.isel(**slab_indices).fillna(0.0).astype(np.float32)

In [ ]:
v_C_slab = xr.zeros_like(v_B_slab)
v_C_slab = v_C_slab.rename({'xu_ocean': 'xv_c'})
v_C_slab.coords['xv_c'] = xv_c
v_C_slab = v_C_slab.rename({'yu_ocean': 'yv_c'})
v_C_slab.coords['yv_c'] = yv_c
v_C_slab.attrs['long_name'] = 'j-surface current on SW convention C grid'


# 	for i in range(LON_POINTS):
# 	    for j in range(LAT_POINTS):
# 	        v_C[i,j] = (v_B[i-1, j-1] + v_B[i, j-1])/2
            
roll_1 = {'xu_ocean': 1, 'yu_ocean': 1}
roll_2 = {'xu_ocean': 0, 'yu_ocean': 1}

v_C_slab.values = (roll_values(v_B_slab, roll_1).values + roll_values(v_B_slab, roll_2).values)/2

In [ ]:
vel_C = xr.merge([u_C_slab, v_C_slab])

In [ ]:
vel_C.to_netcdf(out+str(year)+'_depth_vel_C_gridded.nc')

In [2]:
ds = xr.open_dataset('/archive/net/CM2.6/regrid_annual/regrid_depth_/202_depth_vel_C_gridded.nc')

/nbhome/net/anaconda2/lib/python2.7/site-packages/xarray/conventions.py:389: RuntimeWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using dummy netCDF4.datetime objects instead, reason: dates out of range
  result = decode_cf_datetime(example_value, units, calendar)
/nbhome/net/anaconda2/lib/python2.7/site-packages/xarray/conventions.py:408: RuntimeWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using dummy netCDF4.datetime objects instead, reason: dates out of range
  calendar=self.calendar)
